In [55]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [57]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [58]:
import spacy

In [59]:
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [60]:
from nltk.corpus import stopwords

In [61]:
stop_words = stopwords.words('english')

In [62]:
stop_words = stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [75]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


In [76]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [77]:
data_words = list(sent_to_words(data))

In [78]:
print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [79]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [80]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [81]:
def remove_stopwords(texts):
    print(texts)
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [82]:
def make_bigrams(text):
  return [bigram_mod[doc] for doc in text]

In [83]:
def make_trigrams(text):
  return [trigram_mod[bigram_mod[doc]] for doc in text]

In [84]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV', 'VERB']):
  texts_out = []
  for sentence in texts:
    doc = nlp(" ".join(sentence))
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return texts_out

In [85]:
data_words = list(sent_to_words(data))

In [86]:
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [89]:
data_words_nostops = remove_stopwords(data_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: ignored

In [91]:
data_words_bigrams = make_bigrams(data_words)

In [92]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [93]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADV', 'ADJ', 'VERB'])

In [94]:
id2word = corpora.Dictionary(data_lemmatized)

In [95]:
texts = data_lemmatized

In [96]:
corpus = [id2word.doc2bow(text) for text in texts]

In [97]:
corpus[:1]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 5),
  (6, 1),
  (7, 1),
  (8, 2),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1)]]

In [98]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [99]:
lda_model.print_topics()

[(0,
  '0.020*"physical" + 0.017*"patient" + 0.010*"sin" + 0.009*"drug" + 0.009*"study" + 0.008*"pain" + 0.008*"direct" + 0.008*"disease" + 0.007*"treatment" + 0.007*"tv"'),
 (1,
  '0.015*"people" + 0.012*"can" + 0.011*"say" + 0.011*"other" + 0.009*"will" + 0.009*"would" + 0.009*"may" + 0.009*"believe" + 0.008*"reason" + 0.008*"only"'),
 (2,
  '0.027*"use" + 0.017*"will" + 0.016*"key" + 0.013*"mail" + 0.011*"information" + 0.011*"system" + 0.010*"send" + 0.010*"include" + 0.009*"new" + 0.008*"chip"'),
 (3,
  '0.020*"entry" + 0.013*"format" + 0.013*"mount" + 0.011*"ground" + 0.011*"route" + 0.008*"rational" + 0.008*"title" + 0.008*"lead" + 0.008*"power" + 0.007*"battery"'),
 (4,
  '0.025*"line" + 0.021*"would" + 0.020*"write" + 0.018*"subject" + 0.015*"article" + 0.013*"can" + 0.012*"be" + 0.012*"know" + 0.012*"go" + 0.011*"will"'),
 (5,
  '0.010*"kill" + 0.008*"people" + 0.007*"say" + 0.007*"war" + 0.007*"attack" + 0.007*"gun" + 0.007*"israeli" + 0.007*"fire" + 0.006*"soldier" + 0.006*

In [100]:
lda_model.log_perplexity(corpus)

-8.113111246885369

In [101]:
coherence_model = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [102]:
coherence_model.get_coherence()

0.4408532936256965